 **Point Cloud Encoder**

 *Importing Libraries*

In [1]:
# %%
import torch
import open3d as o3d
import pandas as pd
import numpy as np
from itertools import product
import os
from code_snippets.readers import read_points_file
from itertools import product
import torchsparse
from torch import nn
import torchsparse.nn as spnn
import torchsparse.nn.functional as F
import torchsparse.utils as sp_utils
from torchsparse import SparseTensor
from torchsparse.utils.quantize import sparse_quantize




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


 *Initializing the Point Clouds Data*

In [2]:
# %%
def read_points_file(filepath):
    assert os.path.exists(filepath), f"Could not find point cloud file: {filepath}"
    df = pd.read_csv(filepath, compression="gzip")
    point_cloud = df[["px_world", "py_world", "pz_world"]]
    dist_std = df["dist_std"]
    print(f"Loaded point cloud with {len(point_cloud)} points.")
    return point_cloud.to_numpy(), dist_std.to_numpy()

# Example usage
points, dist_std = read_points_file("/home/mseleem/Desktop/3d_model_pt/0/semidense_points.csv.gz")


Loaded point cloud with 433426 points.


 *Voxelize Point Clouds*

In [3]:
# %%
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)


In [4]:
# %%
num_points = len(pcd.points)
print(f"Number of points: {num_points}")


Number of points: 433426


In [4]:
# %%
voxel_size = 0.015  # 1.5cm
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)


In [6]:
# %%
print("Number of voxels:", len(voxel_grid.get_voxels()))


Number of voxels: 331598


In [5]:
# %%
o3d.visualization.draw_geometries([voxel_grid])


In [8]:
# %%
# Initialize a dictionary to map voxel indices to point indices
voxel_map = {}

# Map points to their respective voxels
for i, point in enumerate(points):
    voxel_index = tuple(voxel_grid.get_voxel(point))
    if voxel_index in voxel_map:
        voxel_map[voxel_index].append(i)
    else:
        voxel_map[voxel_index] = [i]


In [9]:
# %%
# Check for empty voxels
empty_voxels = [k for k, v in voxel_map.items() if len(v) == 0]
assert len(empty_voxels) == 0, "There are empty voxels in the mapping!"
print("All voxels have associated points.")


All voxels have associated points.


In [10]:
# %%
# Check total points
total_points_in_voxels = sum(len(v) for v in voxel_map.values())
print(f"Total points in voxels: {total_points_in_voxels}")
print(f"Total original points: {len(points)}")
assert total_points_in_voxels == len(points), f"Mismatch in point counts: {total_points_in_voxels} != {len(points)}"


Total points in voxels: 433426
Total original points: 433426


In [11]:
# %%
# Aggregate features for each voxel
aggregated_features = []
filtered_voxel_indices = []
for voxel_index, point_indices in voxel_map.items():
    aggregated_feature = np.mean(dist_std[point_indices])  # Example aggregation: mean
    aggregated_features.append(aggregated_feature)
    filtered_voxel_indices.append(voxel_index)


In [12]:
# %%
# Print a few voxel mappings for verification
print("Sample voxel mappings (first 10):")
for k, v in list(voxel_map.items())[:10]:
    print(f"Voxel index: {k}, Point indices: {v}")


Sample voxel mappings (first 10):
Voxel index: (933, 623, 427), Point indices: [0, 268155]
Voxel index: (969, 567, 399), Point indices: [1]
Voxel index: (977, 571, 403), Point indices: [2]
Voxel index: (1003, 533, 396), Point indices: [3]
Voxel index: (1006, 534, 400), Point indices: [4]
Voxel index: (955, 620, 440), Point indices: [5]
Voxel index: (961, 611, 428), Point indices: [6]
Voxel index: (994, 564, 410), Point indices: [7]
Voxel index: (988, 577, 591), Point indices: [8]
Voxel index: (968, 611, 428), Point indices: [9]


In [13]:
# %%
# Verify feature aggregation for a few voxels
print("\nAggregated features for sample voxels (first 10):")
aggregated_features = []
filtered_voxel_indices = []
for voxel_index, point_indices in list(voxel_map.items())[:10]:  # Check first 10 voxels
    aggregated_feature = np.mean(dist_std[point_indices])  # Example aggregation: mean
    print(f"Voxel index: {voxel_index}, Aggregated feature: {aggregated_feature:.4f}")
    aggregated_features.append(aggregated_feature)
    filtered_voxel_indices.append(voxel_index)



Aggregated features for sample voxels (first 10):
Voxel index: (933, 623, 427), Aggregated feature: 0.0022
Voxel index: (969, 567, 399), Aggregated feature: 0.0140
Voxel index: (977, 571, 403), Aggregated feature: 0.0179
Voxel index: (1003, 533, 396), Aggregated feature: 0.0358
Voxel index: (1006, 534, 400), Aggregated feature: 0.0357
Voxel index: (955, 620, 440), Aggregated feature: 0.0023
Voxel index: (961, 611, 428), Aggregated feature: 0.0040
Voxel index: (994, 564, 410), Aggregated feature: 0.0155
Voxel index: (988, 577, 591), Aggregated feature: 0.0350
Voxel index: (968, 611, 428), Aggregated feature: 0.0044


In [14]:
# %%
# Convert to tensor (for the full dataset)
all_aggregated_features = []
all_filtered_voxel_indices = []
for voxel_index, point_indices in voxel_map.items():
    aggregated_feature = np.mean(dist_std[point_indices])
    all_aggregated_features.append(aggregated_feature)
    all_filtered_voxel_indices.append(voxel_index)


In [15]:
# %%
# Convert to tensor
voxel_indices_tensor = torch.tensor(all_filtered_voxel_indices, dtype=torch.int32)
features_tensor = torch.tensor(all_aggregated_features, dtype=torch.float32).view(-1, 1)


In [16]:
# %%
# Add batch dimension to voxel indices
batch_indices = torch.zeros((voxel_indices_tensor.shape[0], 1), dtype=torch.int32)
voxel_indices_tensor_with_batch = torch.cat([batch_indices, voxel_indices_tensor], dim=1)


In [17]:
# %%
# Print dimensions of the tensors
print(f"Voxel indices tensor dimensions: {voxel_indices_tensor_with_batch.shape}")
print(f"Features tensor dimensions: {features_tensor.shape}")


Voxel indices tensor dimensions: torch.Size([331598, 4])
Features tensor dimensions: torch.Size([331598, 1])


In [18]:
# %%
# Create sparse tensor
sparse_tensor = SparseTensor(features_tensor, voxel_indices_tensor_with_batch)

# Print dimensions of the sparse tensor
print(f"Sparse tensor feature dimensions: {sparse_tensor.F.shape}")
print(f"Sparse tensor coordinate dimensions: {sparse_tensor.C.shape}")


Sparse tensor feature dimensions: torch.Size([331598, 1])
Sparse tensor coordinate dimensions: torch.Size([331598, 4])


In [20]:
# %%
class SparseResNetEncoder(nn.Module):
    def __init__(self):
        super(SparseResNetEncoder, self).__init__()
        self.conv1 = spnn.Conv3d(1, 16, kernel_size=3, stride=2)
        self.conv2 = spnn.Conv3d(16, 32, kernel_size=3, stride=2)
        self.conv3 = spnn.Conv3d(32, 64, kernel_size=3, stride=2)
        self.conv4 = spnn.Conv3d(64, 128, kernel_size=3, stride=2)
        self.conv5 = spnn.Conv3d(128, 512, kernel_size=3, stride=2)

    def forward(self, x):
        print(f"Input to conv1: {x.F.shape}, {x.C.shape}")
        x = self.conv1(x)
        print(f"Output of conv1: {x.F.shape}, {x.C.shape}")
        x = self.conv2(x)
        print(f"Output of conv2: {x.F.shape}, {x.C.shape}")
        x = self.conv3(x)
        print(f"Output of conv3: {x.F.shape}, {x.C.shape}")
        x = self.conv4(x)
        print(f"Output of conv4: {x.F.shape}, {x.C.shape}")
        x = self.conv5(x)
        print(f"Output of conv5: {x.F.shape}, {x.C.shape}")
        return x



In [23]:
# %%
# Initialize the model
encoder = SparseResNetEncoder()

# Forward pass
encoded_features = encoder(sparse_tensor)


# Append coordinates to feature vectors for positional encoding
# coordinates = sparse_tensor.C.float()
# encoded_features_with_pos = torch.cat([encoded_features.F, coordinates], dim=1)

# Sort the feature vectors lexicographically by coordinates
# sorted_indices = torch.lexsort((coordinates[:, 2], coordinates[:, 1], coordinates[:, 0]))
# sorted_features_with_pos = encoded_features_with_pos[sorted_indices]

# print("Encoded and sorted features with positional encoding:")
# print(sorted_features_with_pos)




Input to conv1: torch.Size([331598, 1]), torch.Size([331598, 4])
RuntimeError during encoder forward pass: 
